In [1]:
from py_env_train import *
import argparse

leadtime="day02"
HPT_path="HPT/"
mask_type = "no_na"
LR=0.001
BS=16
lr_factor=0.5
Filters=64

# Define the data specifications:
model_data = ["ADAPTER_DE05."+ leadtime + ".merged.nc"]
reference_data = ["ADAPTER_DE05.day01.merged.nc"]
task_name = "spatiotemporal"
mm = "MM"  # or DM
date_start = "2018-01-01T13"
date_end = "2023-12-31T23"
variable = "pr"
laginensemble = False
min_delta_or_lr=0.00000000000001 #just to avoid any limitations

# Define the following for network configs:
loss = "mse"
min_LR = min_delta_or_lr
lr_patience = 2
patience = 8
epochs = 64
val_split = 0.25
n_channels = 7
xpixels = 128
ypixels = 256

filename = func_train.data_unique_name_generator(model_data, reference_data, task_name, mm, date_start, date_end, variable, mask_type, laginensemble)
data_unique_name = filename[:-4]
print(data_unique_name)

# load the training data
print("Loading training data...")
train_files = np.load(TRAIN_FILES + "/" + filename)

train_x = train_files["train_x"]
train_y = train_files["train_y"]
#train_m = train_files["train_m"]
val_x = train_files["val_x"]
val_y = train_files["val_y"]
#val_m = train_files["val_m"]
print("Data loaded!")

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(BS)
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(BS)

train_x, train_y, val_x, val_y = None, None, None, None

training_unique_name = func_train.generate_training_unique_name(loss, Filters, LR, min_LR, lr_factor, lr_patience, BS, patience, val_split, epochs)
print(training_unique_name)

model = func_train.UNET_ATT(xpixels, ypixels, n_channels, Filters)
optimizer = tf.keras.optimizers.Adam(learning_rate=LR, name='Adam')
model.compile(optimizer=optimizer, loss=loss)

# Define the model checkpoint and early stopping callbacks
model_path = PPROJECT_DIR2 + HPT_path + training_unique_name + '_' + leadtime + '.h5'
checkpointer = tf.keras.callbacks.ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor='val_loss')
callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss')]

# Define the ReduceLROnPlateau callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=lr_factor, patience=lr_patience, min_lr=min_LR, min_delta=min_delta_or_lr)

print("Training the model...")

# Train the model using train_dataset and val_dataset
results = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, verbose=2, callbacks=[callbacks, checkpointer, reduce_lr])

# Save and plot the results
print("Saving and plotting the results...")
RESULTS_DF = pd.DataFrame(results.history)
RESULTS_DF.to_csv(PPROJECT_DIR2 + HPT_path + training_unique_name + "_" + leadtime + ".csv")

train_data_hourly_pr_['ADAPTER_DE05.day02.merged.nc']_['ADAPTER_DE05.day01.merged.nc']_MM_6__spatiotemporal_128.256_2018-01-01T13_2023-12-31T23_no_na
Loading training data...
Data loaded!


2024-02-29 14:43:08.258495: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38252 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2024-02-29 14:43:08.317010: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38252 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:44:00.0, compute capability: 8.0
2024-02-29 14:43:08.318784: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38252 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:84:00.0, compute capability: 8.0
2024-02-29 14:43:08.320688: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 38252 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-40GB, pci bu

mse_64_0.001_1e-14_0.5_2_16_8_0.25_64
Training the model...
Epoch 1/64


2024-02-29 14:43:29.993951: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2024-02-29 14:43:35.734627: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8301


2463/2463 - 229s - loss: 0.0888 - val_loss: 0.0738

Epoch 00001: val_loss improved from inf to 0.07380, saving model to /p/project/cesmtst/patakchiyousefi1/HPT/mse_64_0.001_1e-14_0.5_2_16_8_0.25_64_day02.h5


/p/software/juwelsbooster/stages/2022/software/TensorFlow/2.6.0-gcccoremkl-11.2.0-2021.4.0-CUDA-11.5/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/64


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/p/software/juwelsbooster/stages/2022/software/Python/3.9.6-GCCcore-11.2.0/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3578/675679135.py", line 73, in <module>
    results = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, verbose=2, callbacks=[callbacks, checkpointer, reduce_lr])
  File "/p/software/juwelsbooster/stages/2022/software/TensorFlow/2.6.0-gcccoremkl-11.2.0-2021.4.0-CUDA-11.5/lib/python3.9/site-packages/keras/engine/training.py", line 1184, in fit
    tmp_logs = self.train_function(iterator)
  File "/p/software/juwelsbooster/stages/2022/software/TensorFlow/2.6.0-gcccoremkl-11.2.0-2021.4.0-CUDA-11.5/lib/python3.9/site-packages/tensorflow/python/eager/def_function.py", line 885, in __call__
    result = self._call(*args, **kwds)
  File "/p/software/juwelsbooster/stages/2022/software/TensorFlow/2.6

TypeError: object of type 'NoneType' has no len()